In [0]:
%run ./init

In [0]:
# Create a text widget for manually inserting the directory name
dbutils.widgets.text("data_flow_insertion", "", "Enter Directory Name (e.g., DF1)")
data_flow_sel = dbutils.widgets.get("data_flow_insertion")
data_flow_sel = data_flow_sel.upper()
print(data_flow_sel)

In [0]:

# Query the configuration table to get distinct source systems
getSourceInfo = f"""
    SELECT DISTINCT SourceSystem FROM codeconverter_config.DataFlowTableInfo
    WHERE DataFlow = '{data_flow_sel}'
"""
source_systems = spark.sql(getSourceInfo).collect()

# Extract the source system names
source_systems_list = [row['SourceSystem'] for row in source_systems]

# Determine the flag based on the source systems found
if len(source_systems_list) == 1:
    source_system_flag = source_systems_list[0]  # Either 'Hana' or 'BW'
elif len(source_systems_list) == 2:
    source_system_flag = "HANA/BW"
else:
    source_system_flag = "None"  # In case there's no valid source system


# Set a very large timeout to avoid timing out
timeout_seconds = 9999999  # A very large value (effectively no timeout)

# Based on the flag, trigger the corresponding notebook(s) without arguments or timeout
if source_system_flag == "HANA":
    print("Only HANA files found, triggering HANA parsing notebook.")
    dbutils.notebook.run("./CodeConvertFromXMLtoSQL", timeout_seconds=timeout_seconds)

elif source_system_flag == "BW":
    print("Only BW files found, triggering BW parsing notebook.")
    dbutils.notebook.run("./CodeConvertFromHTMLtoSQL", timeout_seconds=timeout_seconds)

elif source_system_flag == "HANA/BW":
    print("Both HANA and BW files found, triggering both parsing notebooks.")
    
    #Trigger Hana parsing notebook
    dbutils.notebook.run("./CodeConvertFromXMLtoSQL", timeout_seconds=timeout_seconds)
    
    #Trigger BW parsing notebook
    dbutils.notebook.run("./CodeConvertFromHTMLtoSQL", timeout_seconds=timeout_seconds)

else:
    print("No valid source system found. Aborting.")